In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount = 'True')

Mounted at /content/drive/


In [ ]:
import numpy as np
import pandas as pd

import re
import string
import csv
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM, Dense, Input, Embedding, Concatenate, TimeDistributed, Bidirectional, GRU
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from keras.utils.vis_utils import plot_model
! pip install rouge
from rouge import Rouge
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


# Importing package and summarizer for TextRank
import gensim
from gensim.summarization import summarize

# Importing the sumy package
!pip install sumy
import sumy
# Importing the parser and tokenizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
# Importing the LexRank summarizer
from sumy.summarizers.lex_rank import LexRankSummarizer

# Importing the Lsa summarizer
from sumy.summarizers.lsa import LsaSummarizer

# Importing the Luhn summarizer
from sumy.summarizers.luhn import LuhnSummarizer

# Importing the KL-Sum summarizer
from sumy.summarizers.kl import KLSummarizer




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 97 kB 3.5 MB/s 
     |████████████████████████████████| 10.1 MB 46.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21714 sha256=24e9ab4c7afd3ae2ff0e52f6f682762359d525bb496f458984ee6fa104b1891b
  Stored in directory: /root/.cache/pip/wheels/5f/0d/0c/2062d8c1758b4b1a2e42b4a63e6660d9ec2ba9463cfee9eeab
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=6243b47ed6c2061c05a8406011d985244e9986fa0868b45d826219a2d21d66dc
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681845 sha256=8da0052dab1994252f815af58e

In [ ]:
#test csv
test_df = pd.read_csv('/content/drive/MyDrive/Statistical Software - MSDS Fall 2022/Data/test.csv')

In [ ]:
print('test:', test_df.shape)

test: (11490, 3)


In [ ]:
test_df.head()

,id,article,highlights
0,92c514c913c0bdfe25341af9fd72b29db544099b,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,2003841c7dc0e7c5b1a248f9cd536d727f27a45a,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,91b7d2311527f5c2b63a65ca98d21d9c92485149,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...
3,caabf9cbdf96eb1410295a673e953d304391bfbb,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...
4,3da746a7d9afcaa659088c8366ef6347fe6b53ea,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6..."


## Checking Null Values

## Checking Duplicates

In [ ]:
test_df = test_df.drop(['id'], axis = 1)
test_df = test_df.reset_index(drop=True)

# Applying Extractive Summarization
### To get smaller inputs for seq2seq model and only using the most important sentences.

In [ ]:
# Taking only 500 instances of testing data since we used the same for intermediate report.
small_df = pd.DataFrame()
small_df['text'] = test_df['article'][:500]
small_df['summary'] = test_df['highlights'][:500]

In [ ]:
small_df.head()

,text,summary
0,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...
3,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...
4,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6..."


In [ ]:
small_df['text'][0]

"Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans. 'In a world where animals have more rights to space and food than humans,' said Charlie Leocha, consumer representative on the committee.\xa0'It is time that the DOT and FAA take a stand for humane treatment of passengers.' But could crowding on planes lead to more serious issues than fighting for sp

In [ ]:
import nltk
nltk.download('punkt')
nltk_generated_summaries=[]

from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser

# tokenize the text:
for i in range(len(small_df)):
  parser=PlaintextParser.from_string(small_df['text'][i],Tokenizer('english'))
  # Instantiating the LsaSummarizer()
  lsa_summarizer=LsaSummarizer()
  lsa_summary= lsa_summarizer(parser.document,3)
  # Printing the summary
  summary = ' '
  for sentence in lsa_summary:
      summary += " " + str(sentence)
  print(i)
  nltk_generated_summaries.append(summary)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
df_extractive_generated = pd.DataFrame(columns=['article', 'summary_generated'])
df_extractive_generated['article']=small_df['text']
df_extractive_generated['summary_generated']=nltk_generated_summaries
df_extractive_generated.head()

,article,summary_generated
0,Ever noticed how plane seats appear to be gett...,With increasing numbers of people taking to ...
1,A drunk teenage boy had to be rescued by secur...,Mr Kumar explained afterwards that he was dr...
2,Dougie Freedman is on the verge of agreeing a ...,Dougie Freedman is on the verge of agreeing ...
3,Liverpool target Neto is also wanted by PSG an...,Neto rushes from his goal as Juan Iturbe bea...
4,Bruce Jenner will break his silence in a two-h...,Bruce Jenner will break his silence in a two...


In [ ]:
with open('/content/drive/MyDrive/Statistical Software - MSDS Fall 2022/Data/result_extractive_sumy_lsa.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['Article', 'Original Summary', 'Model Output'])
    for i in range(500):
        writer.writerow([df_extractive_generated.article[i], small_df.summary[i], df_extractive_generated.summary_generated[i]])

In [ ]:
rouge = Rouge()
rouge.get_scores(nltk_generated_summaries, small_df['summary'], avg=True, ignore_empty=True)

{'rouge-1': {'r': 0.32210567517813055,
  'p': 0.23957170793687943,
  'f': 0.26614072363767627},
 'rouge-2': {'r': 0.09475475658896292,
  'p': 0.07187144008069599,
  'f': 0.07829128092827836},
 'rouge-l': {'r': 0.29001369597109206,
  'p': 0.21600788053466236,
  'f': 0.23976917509841317}}